In [81]:
import numpy as np 
import pandas as pd 
import sklearn
import warnings

In [82]:
def warn(*args, **kwargs):
    pass 
warnings.warn = warn

In [83]:
# file csv yang akan diproses
file_2 = 'lampu.csv'

# panggil filenya dan masukan ke dalam variabel data_n
data_2 = pd.read_csv(file_2)

# lihat dimensi semua filenya 
print('(n_baris, n_kolom)')
print('data_2 :\n', data_2.shape)

(n_baris, n_kolom)
data_2 :
 (10, 7)


### DATA_1

In [84]:
data_2

,Bohlam,Timah Soket,Kabel Lampu,Saklar Lampu,Pengapian,Regulator,Hasil
0,Bagus,Bagus,Bagus,Bagus,Bagus,Bagus,Tidak ada yang diganti
1,Putus,Bagus,Bagus,Bagus,Bagus,Bagus,Ganti bohlam
2,Bagus,Meleleh,Bagus,Bagus,Bagus,Bagus,Ganti timah soket
3,Bagus,Bagus,Putus,Bagus,Bagus,Bagus,Ganti kabel lampu
4,Bagus,Bagus,Bagus,Rusak,Bagus,Bagus,Ganti saklar lampu
5,Bagus,Bagus,Bagus,Bagus,Rusak,Bagus,Ganti pengapian
6,Bagus,Bagus,Bagus,Bagus,Bagus,Rusak,Ganti regulator
7,Putus,Meleleh,Bagus,Bagus,Bagus,Bagus,Ganti bohlam -> timah soket
8,Putus,Meleleh,Putus,Bagus,Bagus,Bagus,Ganti bohlam -> timah soket -> kabel
9,Putus,Meleleh,Putus,Rusak,Bagus,Bagus,Ganti bohlam -> timah soket -> kabel->saklar


In [85]:
# kita lihat nilai - nilai unique dalam datanya
data_2['Hasil'].unique()

array(['Tidak ada yang diganti', 'Ganti bohlam', 'Ganti timah soket',
       'Ganti kabel lampu', 'Ganti saklar lampu', 'Ganti pengapian',
       'Ganti regulator', 'Ganti bohlam -> timah soket',
       'Ganti bohlam -> timah soket -> kabel',
       'Ganti bohlam -> timah soket -> kabel->saklar'], dtype=object)

In [86]:
# kita hitung berapa jumlah kelas di kolom 'Hasil' tersbeut 
data_2['Hasil'].value_counts()

Ganti pengapian                                 1
Ganti saklar lampu                              1
Ganti bohlam -> timah soket -> kabel            1
Ganti bohlam -> timah soket                     1
Tidak ada yang diganti                          1
Ganti bohlam -> timah soket -> kabel->saklar    1
Ganti kabel lampu                               1
Ganti regulator                                 1
Ganti timah soket                               1
Ganti bohlam                                    1
Name: Hasil, dtype: int64

Terlihat bahwa jumlah class yang tersebar di dalam kolom Hasil itu bersifat tidak merata. artinya class 'Tidak ada yang diganti' memiliki jumlah paling banyak (mendominasi) dibandingkan dengan class yang lain seperti 'Oli diganti', 'tali gas diganti' dll. sehingga, agar menghasilkan model yang baik, maka jumlah class ini harus disama-ratakan (balancing) agar distribusi nilainya merata. sehingga akan semakin baik dalam menyimpulkan/memprediksi data

In [87]:
# Perbanyak datasetnya
data_2_rev = np.concatenate([data_2.values, data_2.values, data_2.values])
data_2_rev = pd.DataFrame(data_2_rev)

# perbaiki nama_kolomnya
data_2_rev.columns = data_2.columns
data_2_rev.head()

,Bohlam,Timah Soket,Kabel Lampu,Saklar Lampu,Pengapian,Regulator,Hasil
0,Bagus,Bagus,Bagus,Bagus,Bagus,Bagus,Tidak ada yang diganti
1,Putus,Bagus,Bagus,Bagus,Bagus,Bagus,Ganti bohlam
2,Bagus,Meleleh,Bagus,Bagus,Bagus,Bagus,Ganti timah soket
3,Bagus,Bagus,Putus,Bagus,Bagus,Bagus,Ganti kabel lampu
4,Bagus,Bagus,Bagus,Rusak,Bagus,Bagus,Ganti saklar lampu


### Random oversampling

In [88]:
'''
import library imblearn. karena data kita mengalami imbalance class
di mana imbalance class ini merupakan kondisi ketidakseimabagan jumlah 
class yang terdistribusi di dalam suatu field/kolom, sehingga menyebabkan
sistem tersebut mengalami kesulitasn dalam training dan validasi.
'''
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# kita pecah datasetnya menjadi dua bagian yaitu variabel features dan variabel target
# X ==> berisi variabel-variabel features
X = data_2_rev.iloc[:, :-1]

# Y ==> berisi sebuah variabel target
y = data_2_rev.iloc[:, -1]

# mulai sampling
ros = RandomOverSampler(random_state=0, )
X_resampled, y_resampled = ros.fit_sample(X, y)
print('dimensi setelah oversampling')
print('X_resampled\n', X_resampled.shape)
print('y_resampled\n', y_resampled.shape)

dimensi setelah oversampling
X_resampled
 (30, 6)
y_resampled
 (30,)


In [89]:
# lihat hasil setelah oversampling
out = pd.DataFrame({'x_resampled':y})
out['x_resampled'].value_counts()

Ganti pengapian                                 3
Ganti saklar lampu                              3
Ganti bohlam -> timah soket -> kabel            3
Ganti bohlam -> timah soket                     3
Tidak ada yang diganti                          3
Ganti bohlam -> timah soket -> kabel->saklar    3
Ganti kabel lampu                               3
Ganti regulator                                 3
Ganti timah soket                               3
Ganti bohlam                                    3
Name: x_resampled, dtype: int64

Trlihat bahwa jumlah setiap class yang tersebar di kolom hasil memiliki jumlah yang sama yakni sebanyak 4. hal ini berarti variabel target telah memiliki class class yang balance

In [90]:
# REKONSTRUKSI ULANG DATA_1-Nya
data_1_rev = pd.DataFrame(X)
data_1_rev['hasil'] = y
data_1_rev.head()

,Bohlam,Timah Soket,Kabel Lampu,Saklar Lampu,Pengapian,Regulator,hasil
0,Bagus,Bagus,Bagus,Bagus,Bagus,Bagus,Tidak ada yang diganti
1,Putus,Bagus,Bagus,Bagus,Bagus,Bagus,Ganti bohlam
2,Bagus,Meleleh,Bagus,Bagus,Bagus,Bagus,Ganti timah soket
3,Bagus,Bagus,Putus,Bagus,Bagus,Bagus,Ganti kabel lampu
4,Bagus,Bagus,Bagus,Rusak,Bagus,Bagus,Ganti saklar lampu


#### perbaiki nama kolomnya 

In [91]:
list_nama_kolom = data_2.columns # out ==> ['Gas', 'Slongsong Gas', 'Tali Gas', 'Oli', 'Hasil']
data_1_rev.columns = list_nama_kolom
data_1_rev

,Bohlam,Timah Soket,Kabel Lampu,Saklar Lampu,Pengapian,Regulator,Hasil
0,Bagus,Bagus,Bagus,Bagus,Bagus,Bagus,Tidak ada yang diganti
1,Putus,Bagus,Bagus,Bagus,Bagus,Bagus,Ganti bohlam
2,Bagus,Meleleh,Bagus,Bagus,Bagus,Bagus,Ganti timah soket
3,Bagus,Bagus,Putus,Bagus,Bagus,Bagus,Ganti kabel lampu
4,Bagus,Bagus,Bagus,Rusak,Bagus,Bagus,Ganti saklar lampu
5,Bagus,Bagus,Bagus,Bagus,Rusak,Bagus,Ganti pengapian
6,Bagus,Bagus,Bagus,Bagus,Bagus,Rusak,Ganti regulator
7,Putus,Meleleh,Bagus,Bagus,Bagus,Bagus,Ganti bohlam -> timah soket
8,Putus,Meleleh,Putus,Bagus,Bagus,Bagus,Ganti bohlam -> timah soket -> kabel
9,Putus,Meleleh,Putus,Rusak,Bagus,Bagus,Ganti bohlam -> timah soket -> kabel->saklar


### Data Encoding (Transformation)

In [92]:
from sklearn import preprocessing
trans = preprocessing.LabelEncoder()
columns = data_1_rev.columns
data_encoded_storage = {}
labels_storage = {} # ini untuk meta data
for col in columns:
    out = trans.fit_transform(data_1_rev[col])
    out_lbl = trans.classes_
    labels_storage.update({col:out_lbl})
    data_encoded_storage.update({col:out})

### Rekonstruksi kembali dataset data_1_rev-nya ke dalam dataframe

In [93]:
data_1_rev = pd.DataFrame(data_encoded_storage)
data_1_rev.head() # lihat cuman 5 record data saja

,Bohlam,Timah Soket,Kabel Lampu,Saklar Lampu,Pengapian,Regulator,Hasil
0,0,0,0,0,0,0,9
1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,8
3,0,0,1,0,0,0,4
4,0,0,0,1,0,0,7


### Splitting datanya menjadi training dan testing

In [94]:
from sklearn import model_selection
FEATURES = data_1_rev.iloc[:, :-1]
TARGET = data_1_rev.iloc[:, -1]
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    FEATURES, 
    TARGET, 
    test_size=0.2
)
# cek dimensi datanya 
print('x_train : ', x_train.shape)
print('y_train : ', y_train.shape)
print('x_test  : ',  x_test.shape)
print('y_test  : ',  y_test.shape)

x_train :  (24, 6)
y_train :  (24,)
x_test  :  (6, 6)
y_test  :  (6,)


In [95]:
# import library algoritma pelatihannya
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# import library untuk mengukur akurasi dan laporan hasil klasifikasi
from sklearn.metrics import accuracy_score, classification_report

# inisialisasi model-nya
models = {
    "Support_vector_machine": SVC(kernel="linear", gamma="auto"),
    "Decission_Tree": DecisionTreeClassifier(criterion='entropy'),
    "multi_layer_perceptron": MLPClassifier(hidden_layer_sizes=(250),max_iter=1000,batch_size=100)
}

# membuat fungsi untuk melakukan fungsi fitting atau training
def train_model (model,xtrain,ytrain):
    model.fit(xtrain,ytrain)

# membuat fungsi untuk menampilkan akurasi dan classifiation_report
def predict(i,model,xtest,ytest):
    pred = model.predict(xtest)
    accuracy = accuracy_score(ytest, pred) * 100
    print("\n",i,":")
    print(classification_report(ytest, pred))
    print("accuracy : \n", round(accuracy,3),"%\n")

# melakukan training dengan menggunakan semua algoritma yang telah diinisialisasi
for i in models:
    train_model(models[i],x_train,y_train)
    predict(i,models[i],x_test,y_test)


 Support_vector_machine :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         2
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           9       0.33      1.00      0.50         1

   micro avg       0.67      0.67      0.67         6
   macro avg       0.67      0.80      0.70         6
weighted avg       0.56      0.67      0.58         6

accuracy : 
 66.667 %


 Decission_Tree :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         6
   macro avg       1.00      1.00      1.00         6
weighted

In [80]:
labels_storage.get('Hasil')

array(['Ganti bohlam', 'Ganti bohlam -> timah soket',
       'Ganti bohlam -> timah soket -> kabel',
       'Ganti bohlam -> timah soket -> kabel->saklar',
       'Ganti kabel lampu', 'Ganti pengapian', 'Ganti regulator',
       'Ganti saklar lampu', 'Ganti timah soket',
       'Tidak ada yang diganti'], dtype=object)

In [96]:
# kita siimpan model yang sudah dihasilkan ke format pickle (.pkl)
# import library pickle-nya
import pickle
import os 

# ambil models nya
data_no = 2
folder_name = 'model_{}'.format(data_no)

if os.path.exists(folder_name):
    print('path exist')
else:
    os.mkdir(folder_name)

model_mlp = models['multi_layer_perceptron']

# simpan file model yang sudah dilatih ke dalam format pickle
model_name = '{}/model_data_{}.pkl'.format(folder_name, data_no)
pickle.dump(model_mlp, open(model_name, 'wb'))

# simpan variabel class_keputusan ke dalam format pickle
label_name = '{}/label_data_{}.pkl'.format(folder_name, data_no)
pickle.dump(labels_storage, open(label_name, 'wb'))

print('Done')

Done
